# Necessary Packages and Imports

In [1]:
!pip install -q pyqubo==1.4.0
!pip install -q dwave-ocean-sdk==6.10.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.7/18.7 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 k

In [2]:
import math
import time
import random

import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np

from google.colab import drive
from collections import defaultdict
from pyqubo import Spin, Array, Placeholder, Constraint, Binary

# Helper Classes

In [3]:
class Shipment:
    def __init__(self, start, end, volume, has_route=False, route=None):
      self.start = start
      self.end = end
      self.volume = volume
      self.has_route = has_route
      self.route = route

    def __eq__(self, other):
      if isinstance(other, Shipment):
        return (self.start == other.start and
               self.end == other.end and
               self.volume == other.volume and
               self.has_route == other.has_route and
               self.route == other.route)
      return False

    def toString(self):
      return "from " + str(self.start) + " to " + str(self.end) + " carrying " + str(self.volume)

    def __hash__(self):
      return hash((self.start, self.end, self.volume, self.has_route, self.route))

    def get_source(self):
      return self.start

    def get_sink(self):
      return self.end

    def get_vol(self):
      return self.volume

    def assign_route(self, route):
      self.route = route
      self.has_route = True

    def get_route(self):
      if self.has_route == False:
        return None
      else:
        return self.route

In [4]:
class Graph:
  def __init__(self, shipments=None):
    self.graph = defaultdict(dict)  # Adjacency list with weights
    self.shipments = shipments

  def add_edge(self, u, v, weight):
    self.graph[u][v] = weight
    self.graph[v][u] = weight

  def draw_graph(self):
    G = nx.DiGraph()  # Create a directed graph

    # Add nodes and edges with weights as attributes
    for node, neighbors in self.graph.items():
        G.add_node(node)
        for neighbor, weight in neighbors.items():
            G.add_edge(node, neighbor, weight=weight)

    pos = nx.spring_layout(G)  # Use spring layout algorithm

    # Customize node and edge appearance
    nx.draw_networkx_nodes(G, pos, node_size=500, node_color='lightblue')
    edge_labels = nx.get_edge_attributes(G, 'weight')
    nx.draw_networkx_edge_labels(G, pos, edge_labels, font_size=12)
    nx.draw_networkx_edges(G, pos, width=2, alpha=0.7)
    nx.draw_networkx_labels(G, pos, font_size=12, font_color='black')  # Add this line to label nodes

    plt.axis('off')
    plt.show()

  def floyd_warshall(self):
    nodes = list(self.graph.keys())
    num_nodes = len(nodes)
    self.distances = {node: {node: float('inf') for node in nodes} for node in nodes}
    self.paths = {node: {node: None for node in nodes} for node in nodes}

    for node in nodes:
        self.distances[node][node] = 0

    for u in self.graph:
        for v in self.graph[u]:
            self.distances[u][v] = self.graph[u][v]
            self.paths[u][v] = u

    count = 0
    total = len(nodes)**3
    for k in nodes:
        for i in nodes:
            for j in nodes:
                if self.distances[i][j] > self.distances[i][k] + self.distances[k][j]:
                   self.distances[i][j] = self.distances[i][k] + self.distances[k][j]
                   self.paths[i][j] = self.paths[k][j]
                progress = (count + 1) / total * 100
                count += 1
        print(f"\rProgress: {progress:.2f}%", end="")


    return self.distances, self.paths

# Helper Functions

In [5]:
def get_path(paths_dict, start, end):
  if paths_dict[start][end] is None:
     return []

  path = []
  curr = end

  while curr != start:
     pred = paths_dict[start][curr]
     path.append((pred, curr))
     curr = pred

  # path.append(start)
  path.reverse()
  return path

In [6]:
def find_all_paths(s, G, visited=[], path=[], show=False):
  source = s.get_source()
  sink = s.get_sink()

  all_paths = find_all_paths_helper(source, sink, visited, path, [], G, show=show)

  all_paths_edges = []
  for path in all_paths:
    path_edges = []
    for j in range(0, len(path)-1):
      path_edges.append((path[j], path[j+1]))
    all_paths_edges.append(path_edges)

  return all_paths_edges

def find_all_paths_helper(source, sink, visited, path, all_paths, G, max_paths_num=5, show=False, start_time=None, max_time=2):
     # Start the timer if it's not already started
    if start_time is None:
        start_time = time.time()

    if len(all_paths) >= max_paths_num or (time.time() - start_time) > max_time:
        return all_paths

    visited.append(source)
    path.append(source)

    if source == sink:
        all_paths.append(path.copy())
        if show == True:
          print(path)
        if len(all_paths) >= max_paths_num:
            return all_paths
    else:
        for neighbor, weight in G.graph[source].items():
            if neighbor not in visited:
                find_all_paths_helper(neighbor, sink, visited, path, all_paths, G, max_paths_num, show=show, start_time=start_time, max_time=max_time)

    visited.remove(source)
    path.remove(source)

    return all_paths

In [7]:
def R(e, s, G):
  cand_paths = shipments_routes_dict[s]
  e_paths = []
  for path in cand_paths:
    if (e[0], e[1]) in path:
      e_paths.append(path)
  return e_paths

In [8]:
def R_s(e, G):
  all_cand_paths = []
  for s in shipments:
    for path in shipments_routes_dict[s]:
      all_cand_paths.append(path)
  e_paths = []
  for path in all_cand_paths:
    if (e[0], e[1]) in path:
      e_paths.append(path)
  return e_paths

In [9]:
def scheduled(e, s, G):
  if len(R(e, s, G)) > 0:
    return True
  else:
    return False

In [10]:
def d(e):
  (_, _, weight) = e
  return weight

In [11]:
def w(path):
  sum = 0
  for e in path:
    sum += edge_weight_dict[e]
  return sum

In [12]:
def v(s):
  return s.get_vol()

In [13]:
def l(s):
  load = Shipment.get_vol(s)
  return load

# Reading Input File

In [14]:
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
file_path = "/content/drive/MyDrive/Research @ CMU/Shipment Rerouting Algorithms/Networks/Braess_net.tntp"

# Reading the file
with open(file_path, 'r') as file:
    content = file.read()

In [16]:
graph_data = pd.read_csv(file_path, sep='\t', nrows=3)
num_nodes = int(graph_data.loc[0][0][18:])
num_edges = int(graph_data.loc[2][0][18:])

net = pd.read_csv(file_path, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)

df = net

<ipython-input-16-fd33b6c522d0>:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_nodes = int(graph_data.loc[0][0][18:])
<ipython-input-16-fd33b6c522d0>:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  num_edges = int(graph_data.loc[2][0][18:])


# Generate Problem Instance

In [17]:
# Example usage
# set of terminals on G
V = [str(i+1) for i in range(0, num_nodes)]

# set of edges on G (each edge e = (start terminal, ending terminal, weight))
E = []
for index, row in df.iterrows():
  E.append((str(int(row['init_node'])), str(int(row['term_node'])), row['length']))

unweighted_edges = []
for e in E:
  unweighted_edges.append((e[0], e[1]))
  unweighted_edges.append((e[1], e[0]))

weights = []
for e in E:
  weights.append(e[2])
  weights.append(e[2])

edge_weight_dict = dict(zip(unweighted_edges, weights))

# num of trucks per edge
T = [1 for e in unweighted_edges]

# num_truck dict (takes in edge and returns number of trucks on edge)
edge_trucks_dict = dict(zip(unweighted_edges, T))
truck_capacity = 101

G = Graph()

for start, end, weight in E:
  G.add_edge(start, end, weight)

all_edge_truck_dict = dict(zip(E, T))
def t_max(e):
  return all_edge_truck_dict[e]

c_vol = truck_capacity

# G.draw_graph()

# Generating Shipments

In [18]:
# shipments can be provided manually or generated at random

# Manually adding shipments
shipments = [Shipment("1", "4", 50), Shipment("2", "3", 50), Shipment("1", "3", 50), Shipment("2", "4", 50)]

shipments_routes = []
for s in shipments:
    paths = find_all_paths(s, G, [], [])
    shipments_routes.append(paths)

In [19]:
# Automated generation of shipments
# num_shipments =
# shipments = []
# shipments_routes = []
# num_shipments_generated = 0
# total = num_shipments
# while len(shipments) < num_shipments:
#   start = str(random.randint(1, num_nodes))
#   end = str(random.randint(1, num_nodes))
#   while (end == start):
#     end = str(random.randint(1, num_nodes))
#   s = Shipment(start, end, 50)
#   # print(s.toString())
#   paths = find_all_paths(s, G, [], [], show=False)
#   if len(paths) > 0:
#     progress = (num_shipments_generated + 1) / total * 100
#     num_shipments_generated += 1
#     shipments.append(s)
#     shipments_routes.append(paths)
#     print(f"\rProgress: {progress:.2f}%", end="")

In [20]:
shipments_routes_dict = dict(zip(shipments, shipments_routes))

# Setting up the QUBO

In [49]:
def T(e):
  return dictT[e]

In [50]:
def num_used_trucks(e):
  return sum(n*t[e][n] for n in T(e))

In [51]:
def n_max(S):
  return max(S)

In [52]:
def b(s):
  return math.ceil(v(s)*(c_bin/c_vol))

In [53]:
def bin_capacity_slack(e):
  return sum(m*l[e][m] for m in L)

In [54]:
import os
from dwave.system import LeapHybridSampler

In [ ]:
def sortbynumber(str):
    return int(str[2:len(str)-1])

In [58]:
def QUBO(shipments):
  unflattened_all_shipment_routes = []
  for s in shipments:
    # print(shipment_routes_dict[s])
    unflattened_all_shipment_routes.append(shipments_routes_dict[s])

  # setting up binary y_rs
  y_r_arr = {}
  bin_var_yr = Array.create("y", sum(len(shipments_routes_dict[s]) for s in shipments), 'BINARY')
  i = 0
  for s in shipments:
    y_r_s = {}
    routes = shipments_routes_dict[s]
    for r in routes:
      y_r_s[tuple(r)] = bin_var_yr[i]
      i+=1
    y_r_arr[s] = y_r_s

  shipment_route_count_len = sum(len(shipments_routes_dict[s]) for s in shipments)
  shipment_route_indices = [i for i in range(shipment_route_count_len)]
  all_shipment_routes = [item for sublist in unflattened_all_shipment_routes for item in sublist]
  shipment_routes_index_dict = dict(zip(shipment_route_indices, all_shipment_routes))

  # setting up T dictionary for function usage
  dictT = {}
  for e in E:
    dictT[e] = []
    index = 0
    while 2**(index) <= t_max(e):
      dictT[e].append(2**(index))
      index += 1

  # setting up binary t_ens
  t = {}
  bin_var_t = Array.create("t", sum(len(T(e)) for e in E), 'BINARY')
  i = 0
  for e in E:
    # print(e)
    t[e] = {}
    for n in T(e):
      t[e][n] = bin_var_t[i]
      # print(T(e))
      # print(n)
      # print(t[e][n])
      i+=1

  bin_bit_indices = [i for i in range(sum(len(T(e)) for e in E))]
  edge_bit_pairs = []
  for e in E:
    for n in T(e):
      edge_bit_pairs.append((e, n))
  bin_bit_to_edge_bit_dict = dict(zip(bin_bit_indices, edge_bit_pairs))

  # the total truck distance
  H1 = 0
  for e in E:
    sum_nt = 0
    for n in T(e):
      if n == n_max(T(e)):
        new_n = 1 + t_max(e) - n
        sum_nt += new_n*t[e][n]
      else:
        sum_nt += n*t[e][n]
    H1 += d(e) * sum_nt

  H2 = 0
  for s in shipments:
    RS_sum = 0
    RS = shipments_routes_dict[s]
    for r in RS:
      RS_sum += y_r_arr[s][tuple(r)]
    H2 += (RS_sum-1)**2

  # implementing discretized bin capacities
  c_bin = 25

  # defining L
  L = []
  index = 0
  while(2**(index) < c_bin):
    L.append(2**(index))
    index += 1

  # setting up binary l_ens
  l = {}
  bin_var_l = Array.create("l", len(E)*len(L), 'BINARY')
  i = 0
  for e in E:
    l[e] = {}
    for m in L:
      l[e][m] = bin_var_l[i]
      # print((e, m))
      i+=1

  bin_l_indices = [i for i in range(len(E)*len(L))]
  bin_l_e_m = []
  for e in E:
    for m in L:
      bin_l_e_m.append((e, m))
  bin_l_index_to_l_e_m_dict = dict(zip(bin_l_indices, bin_l_e_m))

  H3 = 0
  total = len(E)
  num_edges = 0
  for e in E:
    inner_sum_1 = 0
    for r in R_s(e, G):
      s = None
      for shipment in shipments:
        if r in shipments_routes_dict[shipment]:
          s = shipment
      inner_sum_1 += b(s)*y_r_arr[s][tuple(r)]

    inner_sum_2 = 0
    for m in L:
      inner_sum_2 += m*(l[e][m])

    inner_sum_3 = 0
    for n in T(e):
      if n == n_max(T(e)):
        new_n = 1 + t_max(e) - n
        inner_sum_3 += new_n*t[e][n]
      else:
        inner_sum_3 += n*t[e][n]

    H3 += (inner_sum_1 + inner_sum_2 - c_bin*(inner_sum_3))**2
    num_edges += 1
    percent = 100 * (num_edges / float(total))
    print(f'\rProgress: {percent:.2f}%', end='')

  M = 100000000 # sum is greater than max feasible truck distance

  H = H1 + M*(H2 + H3)

  # Connecting to DWAVE Account
  os.environ['DWAVE_API_TOKEN'] = 'DEV-7aecbd09552afc1cb9cba26c01212e6224025c2c'

  # solving using DWAVE Sampler
  model = H.compile()
  bqm = model.to_bqm()

  sampleKeys = list(sample.sample.keys())
  sampleKeys.sort(key = sortbynumber)
  sortedSample = {i: sample.sample[i] for i in sampleKeys}

  routes_used = []
  t_bin_var_used = []
  l_bin_var_used = []
  for var in sortedSample.keys():
    if var[0] == 'l':
      if sortedSample[var] == 1:
        index = int(var[2:3])
        l_bin_var_used.append(bin_l_index_to_l_e_m_dict[index])
    if var[0] == 't':
      if sortedSample[var] == 1:
        index = int(var[2:3])
        t_bin_var_used.append(bin_bit_to_edge_bit_dict[index])
    if var[0] == 'y':
      if sortedSample[var] == 1:
        index = int(var[2:3])
        routes_used.append(shipment_routes_index_dict[index])

  return routes_used

In [59]:
QUBO(shipments[:32])

Progress: 100.00%

[[('1', '4')],
 [('2', '4'), ('4', '3')],
 [('1', '4'), ('4', '3')],
 [('1', '3'), ('3', '4')]]

# Defining Variables

In [21]:
unflattened_all_shipment_routes = []
for s in shipments:
  # print(shipment_routes_dict[s])
  unflattened_all_shipment_routes.append(shipments_routes_dict[s])

In [22]:
# setting up binary y_rs
y_r_arr = {}
bin_var_yr = Array.create("y", sum(len(shipments_routes_dict[s]) for s in shipments), 'BINARY')
i = 0
for s in shipments:
  y_r_s = {}
  routes = shipments_routes_dict[s]
  for r in routes:
     y_r_s[tuple(r)] = bin_var_yr[i]
     i+=1
  y_r_arr[s] = y_r_s

In [23]:
shipment_route_count_len = sum(len(shipments_routes_dict[s]) for s in shipments)
shipment_route_indices = [i for i in range(shipment_route_count_len)]
all_shipment_routes = [item for sublist in unflattened_all_shipment_routes for item in sublist]
shipment_routes_index_dict = dict(zip(shipment_route_indices, all_shipment_routes))

In [24]:
# setting up T dictionary for function usage
dictT = {}
for e in E:
  dictT[e] = []
  index = 0
  while 2**(index) <= t_max(e):
    dictT[e].append(2**(index))
    index += 1

In [25]:
def T(e):
  return dictT[e]

In [26]:
# setting up binary t_ens
t = {}
bin_var_t = Array.create("t", sum(len(T(e)) for e in E), 'BINARY')
i = 0
for e in E:
  # print(e)
  t[e] = {}
  for n in T(e):
    t[e][n] = bin_var_t[i]
    # print(T(e))
    # print(n)
    # print(t[e][n])
    i+=1

In [27]:
bin_bit_indices = [i for i in range(sum(len(T(e)) for e in E))]
edge_bit_pairs = []
for e in E:
  for n in T(e):
    edge_bit_pairs.append((e, n))
bin_bit_to_edge_bit_dict = dict(zip(bin_bit_indices, edge_bit_pairs))

In [28]:
def num_used_trucks(e):
  return sum(n*t[e][n] for n in T(e))

# Setting up Sums

In [29]:
def n_max(S):
  return max(S)

In [30]:
# the total truck distance
H1 = 0
for e in E:
  sum_nt = 0
  for n in T(e):
    if n == n_max(T(e)):
      new_n = 1 + t_max(e) - n
      sum_nt += new_n*t[e][n]
    else:
      sum_nt += n*t[e][n]
  H1 += d(e) * sum_nt

In [31]:
H2 = 0
for s in shipments:
  RS_sum = 0
  RS = shipments_routes_dict[s]
  for r in RS:
    RS_sum += y_r_arr[s][tuple(r)]
  H2 += (RS_sum-1)**2

In [32]:
# implementing discretized bin capacities
c_bin = 25

# defining L
L = []
index = 0
while(2**(index) < c_bin):
  L.append(2**(index))
  index += 1

In [33]:
def b(s):
  return math.ceil(v(s)*(c_bin/c_vol))

In [34]:
# setting up binary l_ens
l = {}
bin_var_l = Array.create("l", len(E)*len(L), 'BINARY')
i = 0
for e in E:
  l[e] = {}
  for m in L:
    l[e][m] = bin_var_l[i]
    # print((e, m))
    i+=1

In [35]:
bin_l_indices = [i for i in range(len(E)*len(L))]
bin_l_e_m = []
for e in E:
  for m in L:
    bin_l_e_m.append((e, m))
bin_l_index_to_l_e_m_dict = dict(zip(bin_l_indices, bin_l_e_m))

In [36]:
def bin_capacity_slack(e):
  return sum(m*l[e][m] for m in L)

In [37]:
H3 = 0
total = len(E)
num_edges = 0
for e in E:
  inner_sum_1 = 0
  for r in R_s(e, G):
    s = None
    for shipment in shipments:
      if r in shipments_routes_dict[shipment]:
        s = shipment
    inner_sum_1 += b(s)*y_r_arr[s][tuple(r)]

  inner_sum_2 = 0
  for m in L:
    inner_sum_2 += m*(l[e][m])

  inner_sum_3 = 0
  for n in T(e):
    if n == n_max(T(e)):
      new_n = 1 + t_max(e) - n
      inner_sum_3 += new_n*t[e][n]
    else:
      inner_sum_3 += n*t[e][n]

  H3 += (inner_sum_1 + inner_sum_2 - c_bin*(inner_sum_3))**2
  num_edges += 1
  percent = 100 * (num_edges / float(total))
  print(f'\rProgress: {percent:.2f}%', end='')

Progress: 100.00%

In [38]:
M = 100000000 # sum is greater than max feasible truck distance

In [39]:
H = H1 + M*H2 + H3

# Solving the QUBO

In [43]:
# Connecting to DWAVE Account
import os
from dwave.system import LeapHybridSampler
os.environ['DWAVE_API_TOKEN'] = 'DEV-7aecbd09552afc1cb9cba26c01212e6224025c2c'

In [44]:
# solving using DWAVE Sampler
model = H.compile()
bqm = model.to_bqm()

In [45]:
# Getting Results from Sampler
sampler = LeapHybridSampler(solver={'category': 'hybrid'})
sampleset = sampler.sample(bqm)
sample = sampleset.first

In [46]:
def sortbynumber(str):
    return int(str[2:len(str)-1])

In [47]:
sampleKeys = list(sample.sample.keys())
sampleKeys.sort(key = sortbynumber)
sortedSample = {i: sample.sample[i] for i in sampleKeys}

In [48]:
routes_used = []
t_bin_var_used = []
l_bin_var_used = []
for var in sortedSample.keys():
  if var[0] == 'l':
    if sortedSample[var] == 1:
      index = int(var[2:3])
      l_bin_var_used.append(bin_l_index_to_l_e_m_dict[index])
  if var[0] == 't':
    if sortedSample[var] == 1:
      index = int(var[2:3])
      t_bin_var_used.append(bin_bit_to_edge_bit_dict[index])
  if var[0] == 'y':
    if sortedSample[var] == 1:
      index = int(var[2:3])
      routes_used.append(shipment_routes_index_dict[index])
print(routes_used)

[[('1', '4')], [('2', '4'), ('4', '3')], [('1', '4'), ('4', '3')], [('1', '3'), ('3', '4')]]


In [67]:
bqm.num_variables

42

In [68]:
bqm.num_interactions

170